In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp device

# ds18b20

> A module to handle DS18B20 temperature sensors

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import glob
import re
import datetime as dt
from pathlib2 import Path

In [ ]:
#export
def add_one_wire_modules():
    "Add one wire communication device kernel modules to the linux kernel"
    os.system('/usr/sbin/modprobe w1-gpio')
    os.system('/usr/sbin/modprobe w1-therm')

In [ ]:
#export
class DataPoint:
    "TODO"
    def __init__(self, device_sn, date_time, temp_raw, temp_corr = None):
        self.__device_sn = device_sn
        self.__date_time = date_time
        self.__temp_raw = temp_raw
        self.__temp_corr = temp_corr
        if temp_corr is None:
            self.__temp_corr = temp_raw
        
    def get_device_sn(self):
        return self.__device_sn
    
    def get_date_time(self):
        return self.__date_time
    
    def print_date_time(self):
        print(self.__date_time.strftime('%Y-%m-%d %H:%M:%S'))
        
    def get_temp_raw(self):
        return self.__temp_raw
        
    def get_temp_C(self):
        return self.__temp_corr / 1000.0
    
    def get_temp_F(self):
        return self.get_temp_C() * 9.0 / 5.0 + 32.0
    
    def get_temp_K(self):
        return self.get_temp_C() + 273.15

In [ ]:
#export
class Device:
    "TODO"
    def __init__(self, base_dir = '/sys/bus/w1/devices/', device_sn = '28*'):
        self.base_dir = base_dir
        path = Path(base_dir).glob(device_sn + '/w1_slave')
        self.device_files = [x for x in path if x.is_file()]
        self.__raw_low = None
        self.__ref_low = None
        self.__f_corr = None
        self.__cal = {}

    def set_calibration(self, sn, raw_low, raw_high, ref_low, ref_high):
        f_corr = (ref_high - ref_low) / (raw_high - raw_low)
        self.__cal.update({sn:(raw_low, ref_low, f_corr)})
        
    def __read_raw(self, device_file):
        with open(device_file, 'r') as f: 
            rows = f.readlines()
            return rows
        
    def __device_sn(self, device_file):
        dev_path = str(re.findall(r'.*?/(28.*?)/w1_slave', str(device_file))[0])
        return dev_path
    
    def get_temps(self):
        temps = list()
        for df in self.device_files:
            rows = self.__read_raw(df)
            while rows[0].strip()[-3:] != 'YES':
                time.sleep(0.1)
                rows = read_temp_raw()
            pos = rows[1].find('t=')
            if pos != -1:
                device_sn = self.__device_sn(df)
                date_time = dt.datetime.now()
                t_raw = float(rows[1][pos + 2:])
                t_corr = t_raw
                sn = self.__device_sn(df)
                if sn in self.__cal:
                    t_corr = ((t_raw / 1000.0 - self.__cal[sn][0]) * self.__cal[sn][2] + self.__cal[sn][1]) * 1000.0
                data_point = DataPoint(device_sn, date_time, t_raw, t_corr)
                temps.append(data_point)
        return temps

In [ ]:
d = Device(base_dir='resources/')
print(d.get_temps()[0].get_temp_raw(), d.get_temps()[0].get_temp_C())
print(d.get_temps()[1].get_temp_raw(), d.get_temps()[1].get_temp_C())
d.set_calibration('28-03219779d857', -1.563, 97.75, 0.01, 98.7)
print(d.get_temps()[0].get_temp_raw(), d.get_temps()[0].get_temp_C())
print(d.get_temps()[1].get_temp_raw(), d.get_temps()[1].get_temp_C())

20525.0 20.525
20750.0 20.75
20525.0 21.959439851781738
20750.0 20.75
